In [ ]:
import sys
import cmath
import math
import os
import h5py
import matplotlib
import matplotlib.font_manager as fm
from matplotlib.ticker import FormatStrFormatter
import matplotlib.pyplot as plt   # plots
import itertools
import numpy as np
import time
import warnings

from liblibra_core import *
import util.libutil as comn
from libra_py import units
import libra_py.models.Holstein as Holstein
import libra_py.models.Tully as Tully
import libra_py.models.Subotnik as Subotnik
import libra_py.models.Esch_Levine as Esch_Levine
from libra_py import dynamics_plotting

import libra_py.dynamics.tsh.compute as tsh_dynamics
import libra_py.dynamics.tsh.plot as tsh_dynamics_plot
import libra_py.data_savers as data_savers

#from recipes import shxf, mqcxf, ehxf
#from recipes import ehrenfest_adi_ld, ehrenfest_adi_nac, ehrenfest_dia, mfsd
#from recipes import fssh, sdm, bcsh

import libra_py.dynamics.exact.compute as dvr
import libra_py.dynamics.exact.save as dvr_save

#from matplotlib.mlab import griddata
#%matplotlib inline 
warnings.filterwarnings('ignore')

## Color code
colors = {}
colors.update({"11": "#D20C0C", "12": "#FF5339", "13": "#FF8965"}) #red
colors.update({"21": "#00790C", "22": "#49A93F", "23": "#7DDB6D"}) #green
colors.update({"31": "#0009FF", "32": "#7145FF", "33": "#B176FF"}) #blue
colors.update({"41": "#FFA100", "42": "#FFD249", "43": "#FFFF7C"}) #orange
colors.update({"51": "#FF1DCE", "52": "#FFCDFF"}) #magenta
colors.update({"61": "#00ffea", "62": "#70FFFF", "63": "#AFFFFF"}) #cyan
colors.update({"71": "#bd20ff", "72": "#F461FF", "73": "#FF97FF"}) #indigo

clrs_index = ["11", "21", "31", "41", "51", "61", "71", "12", "22", "32", "42", "52", "62", "72" "13","23", "33", "53", "73"]


##  Total energy deviation, coherence, population

In [ ]:
def compute_coherence(file):
    with h5py.File(file, "r") as f:
        C_ADI = f["Cadi/data"][:,:,:]

    nts, ntraj, nst = C_ADI.shape

    # Reshape C_ADI for better performance
    C_ADI_reshaped = C_ADI.reshape(nts, ntraj, nst, 1)

    # Compute rho2 using vectorized operations
    rho2 = np.sum(np.abs(C_ADI_reshaped) ** 2 * np.abs(C_ADI_reshaped.transpose(0, 1, 3, 2)) ** 2, axis=1) / ntraj

    return rho2

In [ ]:
list_td_dirs = ["Subotnik/", "Schwarz_1/", "Schwarz_2/", "Schwarz_3/"]

In [ ]:
dir_coh_temp= list_td_dirs[3] + "data_coherence/"
if not os.path.exists(dir_coh_temp):
    os.mkdir(dir_coh_temp)

In [ ]:
dir_coh="data_coherence/"

In [ ]:
# Compute and save coherence first
#list_methods = [6,7]
#model_indx = 1
#
#for i, _method_indx in enumerate(list_methods):
#    fn = list_td_dirs[3] + F"model{model_indx}-method{_method_indx}-icond0/mem_data.hdf"
#    rho2 = compute_coherence(fn)
#
#    np.savetxt(dir_coh_temp+F"coherence_model{model_indx}_method{_method_indx}_10.txt", rho2[:,1,0])

In [ ]:
def plot_ecp(_model_indx, list_methods = [6, 6, 6, 6, 6], list_dirs = ["../", "Subotnik/", "Schwarz_1/", "Schwarz_2/", "Schwarz_3/"],
             name_methods = ["Fixed", "Subotnik", r"Schwarz, $w^{2}=1$", r"Schwarz, $w^{2}=4$", r"Schwarz, $w^{2}=9$"],
             title=None, markers = ["o"]*5, ms= [6]*5, zorder_methods = [10,11,11,11,11], list_ls=["-", "--", "dotted"],
             _nstates=2, _pad=-1, _bbox_to_anchor="", turn_on_legend=True):

    nst = _nstates

    fig, axs = plt.subplots(3, figsize=(10,12), gridspec_kw=dict(hspace=0), dpi=600)
    nr_mark = 30
    
    if title != None: axs[0].set_title(title, fontsize=28)
    
    # energy deviation
    Hart2eV=27.2114
    axs[0].set_ylabel(r"$|\Delta E|$ (mHa)", fontsize=28)
    axs[0].set_xticks([])
    axs[0].tick_params(axis='both', labelsize=28)
    #axs[0].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    axs[0].margins(x=0)
    
    axs[0].plot([0], [0], label='QD', color='k')
    
    for i, _method_indx in enumerate(list_methods):
        with h5py.File(list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:])
            freq = int(_nstep/nr_mark)
            axs[0].plot(f["time/data"][:]/41.0, np.abs(f["Etot_ave/data"][:]-f["Etot_ave/data"][0])*1000, linewidth=1, 
                label=name_methods[i], color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
    
    if turn_on_legend:
        if _bbox_to_anchor == "":
            axs[0].legend(fontsize=28, ncol=2, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='upper right').set_zorder(100)
        else:
            axs[0].legend(fontsize=28, ncol=2, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='upper right', bbox_to_anchor=_bbox_to_anchor).set_zorder(100)
    
    # coherences
    axs[1].set_ylabel("coherence", fontsize=28)
    axs[1].tick_params(axis='both', labelsize=28)
    axs[1].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    axs[1].margins(x=0)
    axs[1].set_xticks([])
    with h5py.File(F"../exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
        temp = "QD"
    
        cnt=-1
        for ist in range(nst):
            for jst in range(nst):
                if(ist<=jst): continue
                cnt+=1
                axs[1].plot(f["time/data"][:]/41.0, f["coherence_adi/data"][:, ist, jst], 
                            label=temp, linewidth=7, ls=list_ls[cnt], color = 'black', zorder=-10)
                temp = "_nolegend_"
        
        times = f["time/data"][:]
    
    for i, _method_indx in enumerate(list_methods):
        with h5py.File(list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:])
            freq = int(_nstep/nr_mark)
        
        #rho2 = compute_coherence(F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf")
        cnt=-1
        for ist in range(nst):
            for jst in range(nst):
                if(ist<=jst): continue
                cnt+=1
                rho2 = np.loadtxt(list_dirs[i] + dir_coh + F"coherence_model{_model_indx}_method{_method_indx}_{ist}{jst}.txt")
                #axs[2].plot(times/41.0, rho2[:, ist, jst], linewidth=1, ls=list_ls[cnt],
                axs[1].plot(times/41.0, rho2, linewidth=1, ls=list_ls[cnt],
                    color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
    
    # populations
    axs[2].set_xlabel("time (fs)", fontsize=28)
    axs[2].set_ylabel("population", fontsize=28)
    axs[2].margins(x=0)
    axs[2].tick_params(axis='both', labelsize=28)
    #axs[3].set_yticks([0.0, 0.5, 1.0])
    
    with h5py.File(F"../exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
        temp = "QD"
        #for ist in range(nst):
        #    axs[3].plot(f["time/data"][:]/41.0, f["pop_adi/data"][:, ist, 0], label=temp, linewidth=7, color = 'black', zorder=-10)
        axs[2].plot(f["time/data"][:]/41.0, f["pop_adi/data"][:, 1, 0], label=temp, linewidth=7, color = 'black', zorder=-10)
    
    for i, _method_indx in enumerate(list_methods):
        with h5py.File(list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:])
            freq = int(_nstep/nr_mark)
            
            #for ist in range(nst):
            #    axs[3].plot(f["time/data"][:]/41.0, f["se_pop_adi/data"][:, ist], linewidth=1, 
            #        color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
            axs[2].plot(f["time/data"][:]/41.0, f["se_pop_adi/data"][:, 1], linewidth=1, 
                color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

    #fig.tight_layout(pad=_pad)
    fig.align_labels()
    fig.tight_layout()
    #axs[1].legend(fontsize=20)



In [ ]:
def plot_wcp(_model_indx, list_methods = [6, 6, 6, 6, 6], list_dirs = ["../", "Subotnik/", "Schwarz_1/", "Schwarz_2/", "Schwarz_3/"],
             name_methods = ["Fixed", "Subotnik", r"Schwarz, $w^{2}=1$", r"Schwarz, $w^{2}=4$", r"Schwarz, $w^{2}=9$"],
             title=None, markers = ["o"]*5, ms= [6]*5, zorder_methods = [10,11,11,11,11], list_ls=["-", "--", "dotted"],
             _nstates=2, _pad=-1, _bbox_to_anchor="", turn_on_legend=True):

    nst = _nstates
    ntraj = 2000

    fig, axs = plt.subplots(3, figsize=(10,12), gridspec_kw=dict(hspace=0), dpi=600)
    nr_mark = 30
    
    if title != None: axs[0].set_title(title, fontsize=28)
    
    # td width
    axs[0].margins(x=0)
    axs[0].set_ylim([-0.045, 1.545])
    axs[0].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    axs[0].set_ylabel("td width (Bohr)", fontsize=28)
    axs[0].tick_params(axis='both', labelsize=28)
    #with h5py.File(list_dirs[0] + F"model{_model_indx}-method6-icond0/mem_data.hdf", 'r') as f:
    #    axs[0].plot(f["time/data"][1:]/41.0, f["wp_width/data"][1:,0], linewidth=2, 
    #            color = colors["11"], zorder=30)
             
    with h5py.File(list_dirs[1] + F"model{_model_indx}-method{list_methods[1]}-icond0/mem_data.hdf", 'r') as f:
        t = f["time/data"][1:]
        wp_width = np.array(f["wp_width/data"])
        wp_width[wp_width >100] = 0.0
        wp_width[wp_width == 0.0] = np.nan
    
        for it in range(ntraj):
            axs[0].plot(t/41.0, wp_width[1:,it,0], linewidth=0.5, 
                color = colors["21"], zorder=20)
    print(np.nanmean(wp_width), np.nanstd(wp_width))
    #with h5py.File(F"../exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
    #    axs[0].hlines(np.nanmean(wp_width), f["time/data"][1]/41.0, f["time/data"][-1]/41.0, colors=colors["21"], zorder=90 )
    
    axs[0].hlines(0.3, t[1]/41.0, t[-1]/41.0, colors=colors["11"], zorder=90 )
    
    with h5py.File(list_dirs[2] + F"model{_model_indx}-method{list_methods[2]}-icond0/mem_data.hdf", 'r') as f:
        for it in range(ntraj):
            axs[0].plot(f["time/data"][1:]/41.0, np.abs(f["wp_width/data"][1:,it,0]), linewidth=0.5, 
                color = colors["31"], zorder=10)
            
    #with h5py.File(list_dirs[3] + F"model{_model_indx}-method{list_methods[3]}-icond0/mem_data.hdf", 'r') as f:
    #    for it in range(ntraj):
    #        axs[0].plot(f["time/data"][1:]/41.0, np.abs(f["wp_width/data"][1:,it,0]), linewidth=0.5, 
    #            color = colors["41"], zorder=10)
    #        
    #with h5py.File(list_dirs[4] + F"model{_model_indx}-method{list_methods[4]}-icond0/mem_data.hdf", 'r') as f:
    #    for it in range(ntraj):
    #        axs[0].plot(f["time/data"][1:]/41.0, np.abs(f["wp_width/data"][1:,it,0]), linewidth=0.5, 
    #            color = colors["51"], zorder=10)

    # coherences    
    # for labels
    axs[1].plot([], [], color="k", label="QD")
    axs[1].plot([], [], color=colors["11"], label="Fixed", marker=markers[0], markersize=ms[0])
    axs[1].plot([], [], color=colors["21"], label="Subotnik", marker=markers[1], markersize=ms[1])
    
    if turn_on_legend:
        axs[1].legend(fontsize=28, ncol=1, labelspacing = 0.4,  columnspacing=0.5, bbox_to_anchor=(1, 1.05), frameon=False, loc='upper right').set_zorder(100)
    #if turn_on_legend:
    #    if _bbox_to_anchor == "":
    #        axs[0].legend(fontsize=28, ncol=1, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='upper right').set_zorder(100)
    #    else:
    #        axs[0].legend(fontsize=28, ncol=1, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='upper right', bbox_to_anchor=_bbox_to_anchor).set_zorder(100)    

    axs[1].set_ylabel("coherence", fontsize=28)
    axs[1].tick_params(axis='both', labelsize=28)
    axs[1].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    axs[1].margins(x=0)
    axs[1].set_xticks([])
    with h5py.File(F"../exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
        temp = "QD"
    
        cnt=-1
        for ist in range(nst):
            for jst in range(nst):
                if(ist<=jst): continue
                cnt+=1
                axs[1].plot(f["time/data"][:]/41.0, f["coherence_adi/data"][:, ist, jst], 
                            label="", linewidth=7, ls=list_ls[cnt], color = 'black', zorder=-10)
        
        times = f["time/data"][:]
    
    for i, _method_indx in enumerate(list_methods):
        with h5py.File(list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:])
            freq = int(_nstep/nr_mark)
        
        #rho2 = compute_coherence(F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf")
        cnt=-1
        for ist in range(nst):
            for jst in range(nst):
                if(ist<=jst): continue
                cnt+=1
                rho2 = np.loadtxt(list_dirs[i] + dir_coh + F"coherence_model{_model_indx}_method{_method_indx}_{ist}{jst}.txt")
                #axs[2].plot(times/41.0, rho2[:, ist, jst], linewidth=1, ls=list_ls[cnt],
                axs[1].plot(times/41.0, rho2, linewidth=1, ls=list_ls[cnt],
                    color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
    
    # populations
    
    # for label
    axs[2].plot([], [], color=colors["31"], label=r"Schwarz, $w^{2}=1$", marker=markers[2], markersize=ms[2])
    axs[2].plot([], [], color=colors["41"], label=r"Schwarz, $w^{2}=4$", marker=markers[3], markersize=ms[3])
    axs[2].plot([], [], color=colors["51"], label=r"Schwarz, $w^{2}=9$", marker=markers[4], markersize=ms[4])
    
    axs[2].set_xlabel("time (fs)", fontsize=28)
    axs[2].set_ylabel("population", fontsize=28)
    axs[2].margins(x=0)
    axs[2].tick_params(axis='both', labelsize=28)
    #axs[1].set_yticks([0.0, 0.5, 1.0])
    
    with h5py.File(F"../exact-model{_model_indx}-icond0/data.hdf", 'r') as f:
        temp = "QD"
        #for ist in range(nst):
        #    axs[1].plot(f["time/data"][:]/41.0, f["pop_adi/data"][:, ist, 0], label=temp, linewidth=7, color = 'black', zorder=-10)
        axs[2].plot(f["time/data"][:]/41.0, f["pop_adi/data"][:, 1, 0], label="", linewidth=7, color = 'black', zorder=-10)
    
    for i, _method_indx in enumerate(list_methods):
        with h5py.File(list_dirs[i] + F"model{_model_indx}-method{_method_indx}-icond0/mem_data.hdf", 'r') as f:
            _nstep = len(f["time/data"][:])
            freq = int(_nstep/nr_mark)
            
            #for ist in range(nst):
            #    axs[3].plot(f["time/data"][:]/41.0, f["se_pop_adi/data"][:, ist], linewidth=1, 
            #        color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])
            axs[2].plot(f["time/data"][:]/41.0, f["se_pop_adi/data"][:, 1], linewidth=1, 
                color = colors[clrs_index[i]], marker=markers[i], markevery=[x for x in range(0, _nstep, freq)], markersize=ms[i], zorder=zorder_methods[i])

    #if turn_on_legend:
    #    if _bbox_to_anchor == "":
    #        axs[1].legend(fontsize=28, ncol=1, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='lower right').set_zorder(100)
    #    else:
    #        axs[1].legend(fontsize=28, ncol=1, labelspacing = 0.4,  columnspacing=0.5, frameon=False, loc='lower right', bbox_to_anchor=_bbox_to_anchor).set_zorder(100)
    
    if turn_on_legend:
        axs[2].legend(fontsize=28, ncol=1, labelspacing = 0.4,  columnspacing=0.5, frameon=False, bbox_to_anchor=(1,-0.07), loc='lower right').set_zorder(100)
    
    #fig.tight_layout(pad=_pad)
    fig.align_labels()
    fig.tight_layout()
    #axs[1].legend(fontsize=20)



## SHXF

In [ ]:
# ECWR, SHXF
plot_wcp(4, title="ECWR", turn_on_legend=True)

In [ ]:
# DAG, SHXF
plot_wcp(5, title="DAG", turn_on_legend=False)

## MQCXF

In [ ]:
# ECWR, MQCXF
plot_ecp(4, title="ECWR", list_methods = [7]*5, turn_on_legend=True)

In [ ]:
# DAG, MQCXF
plot_ecp(5, title="DAG", list_methods = [7]*5, turn_on_legend=False)